In [304]:
import pandas as pd
import numpy as np

import rows

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import warnings
warnings.filterwarnings('ignore')

from tabula import read_pdf
import camelot

## Itens Necessários para análise
-  orçamento total obrigatório e discricionário
-  alunos matriculados, vagas no vestibular e alunos formandos, tudo aberto por curso, discriminado o número de anos previsto para conclusão

----
## Dados

#### UNICAMP
- [Anuarios](https://www.aeplan.unicamp.br/anuario/anuario.php)

#### USP
- [Orçamento](http://www.usp.br/codage/?q=node/35)
- [Anuarios](https://uspdigital.usp.br/anuario/AnuarioControle)

#### UNESP
- [Anuarios](https://ape.unesp.br/anuario/)

#### Unicamp

- Vestibular
    (1) A partir de 2019 a UNICAMP passou a adotar 4 formas de ingresso (Vestibular, ENEM-UNICAMP, Vestibular Indígena e Vagas Olímpicas). Foram oferecidas 3.396 vaga para ingresso nos cursos de graduação da UNICAMP, distribuídas nos seguintes sistemas de ingresso: 2.589 vagas oferecidas pelo Vestibular, 645 vagas para ENEM-UNICAMP, 23 vagas para Vestibular Indígena (mais 49 vagas adicionais); e 83 vagas para Vagas Olímpicas (mais 7 vagas adicionais). O candidato inscrito está contado uma única vez, através de seu CPF, independente da forma de ingresso e curso escolhido.
- (2) As vagas regulares não preenchidas, do Vestibular Indígena e das Vagas Olímpicas, voltaram a estar disponiveis para preenchimento pelo Vestibular UNICAMP 2019. As vagas adicionais não preenchidas, do Vestibular Indígena e das Vagas Olímpicas, por serem de preenchimento facultativo não foram transferidas para o Vestibular UNICAM 2019.


# Unicamp

## Vestibular

In [19]:
dd = camelot.read_pdf('files/unicamp_anuario_2019.pdf', pages='28-30', flavor='stream', )

In [53]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [61]:
mask = (df[3]!='') & (df[1]!='') & (df[1]!='TOTAL')
df2 = df[mask]

df2[3] = df2[3].str.replace('.','').astype(float)

df2 = df2.drop(columns=[0,9,12,13,14,15])

In [63]:
rename_cols = {
    1:'curso',
    2:'periodo',
    3:'incritos_vestibular',
    4:'incritos_enem',
    5:'incritos_indigena',
    6:'incritos_olimpiada',
    7:'vagas_vestibular',
    8:'vagas_enem',
    10:'vagas_indigena',
    11:'vagas_olimpiada',
    
}

df2 = df2.rename(columns=rename_cols)

In [67]:
df2['ano']=2019

#### 2018 a 2009

In [72]:
dd = camelot.read_pdf('files/unicamp_anuario_2018.pdf', pages='28-34', flavor='stream', )

In [160]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

In [196]:
mask = (df[13]!='') & (df[3]!='Relação Candidatos / Vaga') & (df[4]!='2009')
df2 = df[mask]

df2 = df2.drop(columns=[0])

rename_cols = {
    1:'curso',
    2:'periodo',
    3:'col',
    4:2009,
    5:2010,
    6:2011,
    7:2012,
    8:2013,
    9:2014,
    10:2015,
    11:2016,
    12:2017,
    13:2018
}

df2 = df2.rename(columns=rename_cols)

cursos = df2['curso'].tolist()

curso_final=[]
for i in range(len(cursos)-1):
    if cursos[i]=='':
        curso_final.append(cursos[i])
    else:
        if cursos[i+1]!='':
            curso_final.append(cursos[i] + " " + cursos[i+1])
        else:
            curso_final.append(cursos[i])


curso_final.append(np.nan)

# df2['curso_errado']= df2['curso']
df2['curso']= curso_final

df2 = df2.reset_index(drop=True)
df2 = df2.drop(index=10)
mask = df2[2018]!='Fonte: COMVEST'
df2 = df2[mask]

df2 = df2.reset_index(drop=True)

df2['curso']= np.where(df2.index%2==1, np.nan, df2['curso'])
df2['periodo']= np.where(df2.index%2==1, np.nan, df2['periodo'])

df2['curso'] = df2['curso'].fillna(method='ffill')
df2['periodo'] = df2['periodo'].fillna(method='ffill')

In [252]:
cursos   = df2['curso'].tolist()
periodos = df2['periodo'].tolist()

years = [i+1 for i in range(2008,2018)]

df_final = pd.DataFrame()

for i in range(len(cursos)):
    curso = cursos[i]
    periodo = periodos[i]
#     print(curso)
    for year in years:
        df_f = pd.DataFrame([curso], columns=['curso'])
        df_f['periodo']= periodo
        
        mask = (df2['curso']==curso) & (df2['col']=='Vagas')
        df_f['vagas'] =  df2[mask][year].values[0]
        
        mask = (df2['curso']==curso) & (df2['col']=='Candidatos')
        df_f['candidatos'] =  df2[mask][year].values[0]
        
        df_f['ano'] = year
        
        df_final = pd.concat([df_final,df_f],axis=0)

In [253]:
df_final = df_final.drop_duplicates(subset=df_final.columns).reset_index(drop=True)

In [256]:
# df_final.to_excel('unicamp_vestibular_2009_2018.xlsx', index=False, encoding='utf-8')

### Vagas 2009 - 2018

In [590]:
dd = camelot.read_pdf('files/unicamp_anuario_2019.pdf', pages='48-51', flavor='stream')

In [591]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

# df.loc[83:115] = df.loc[83:115].shift(1,axis=1)
df = df.drop(index=[i for i in range(115,132)], columns=[0])
df.columns = df.loc[0].tolist()


In [597]:
mask = (df[df.columns[0]]!='') & (df['PERÍODO']!='PERÍODO') & (df['PERÍODO']!='')
df2 = df[mask]

cols_rename = {
    df.columns[0]:'curso',
    df.columns[1]:'periodo'
    
}
df2 = df2.rename(columns=cols_rename)

In [598]:
df2['curso'] = df2['curso'].replace(df2['curso'].tolist()[-1], 'Física/Matemática/Mat. Aplicada e Computacional/Engenharia, Física/Física: Médica e Biomédica')
df2['curso'] = df2['curso'].str.replace('\d','', regex=True).str.strip('()').str.strip()

In [600]:
cursos   = df2['curso'].tolist()
periodos = df2['periodo'].tolist()

years = [str(i+1) for i in range(2008,2018)]

df_final = pd.DataFrame()

for i in range(len(cursos)):
    curso = cursos[i]
    periodo = periodos[i]
#     print(curso)
    for year in years:
        df_f = pd.DataFrame([curso], columns=['curso'])
        df_f['periodo']= periodo
        
        mask = (df2['curso']==curso)
        df_f['vagas'] =  df2[mask][year].values[0]
        
        df_f['ano_base'] = year
        
        df_final = pd.concat([df_final,df_f],axis=0)

In [601]:
df_final = df_final.drop_duplicates(subset=df_final.columns).reset_index(drop=True)

In [603]:
# df_final.to_excel('dados/unicamp_vestibular_2009_2018.xlsx', index=False)

## Alunos 2018

In [412]:
dd = camelot.read_pdf('files/unicamp_anuario_2019.pdf', pages='53-59', flavor='stream', )

In [413]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

df.loc[83:115] = df.loc[83:115].shift(1,axis=1)

In [414]:
mask = (df[4]!='')&(df[0]!='UNIDADES') & (df[1]!='OPÇÃO POR INGRESSO')
df2 = df[mask]

In [415]:
df2[1] = np.where(df2[1]=='',df2[0],df2[1] )

df2 = df2[[1,2,3,4,5]]

In [417]:
df2 = df2.drop(index=[84,194,195,196])

In [419]:
df2[1] = df2[1].replace(df2[1].tolist()[-1], 'Física/Matemática/Mat. Aplicada e Computacional/Engenharia, Física/Física: Médica e Biomédica')

In [420]:
cols_rename = {
    1:'curso',
    2:'periodo',
    3:'ingressantes',
    4:'matriculados',
    5:'concluintes'
}
df2 = df2.rename(columns=cols_rename)

df2['ano_base'] = 2018

In [422]:
# df2.to_excel('unicamp_alunos_2018_graduação.xlsx', index=False)

## Alunos 2017

In [423]:
dd = camelot.read_pdf('files/unicamp_anuario_2018.pdf', pages='59-65', flavor='stream', )

In [426]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

df.loc[84:115] = df.loc[84:115].shift(1,axis=1)

In [439]:
mask = (df[4]!='')&(df[0]!='UNIDADES') & (df[1]!='OPÇÃO POR INGRESSO') & (df[2]!='PERÍODO')
df2 = df[mask]

In [440]:
df2[1] = np.where(df2[1]=='',df2[0],df2[1] )

df2 = df2[[1,2,3,4,5]]

In [441]:
df2 = df2.drop(index=[195,196,197])
df2[1] = df2[1].replace(df2[1].tolist()[-1], 'Física/Matemática/Mat. Aplicada e Computacional/Engenharia, Física/Física: Médica e Biomédica')

In [443]:
cols_rename = {
    1:'curso',
    2:'periodo',
    3:'ingressantes',
    4:'matriculados',
    5:'concluintes'
}
df2 = df2.rename(columns=cols_rename)

df2['ano_base'] = 2017

In [445]:
# df2.to_excel('unicamp_alunos_2017_graduação.xlsx', index=False)

## Alunos 2016

In [446]:
dd = camelot.read_pdf('files/unicamp_anuario_2017.pdf', pages='59-65', flavor='stream', )

In [449]:
df = pd.DataFrame()

for i in range(len(dd)):
    df = pd.concat([df,dd[i].df], axis=0)
    
df = df.reset_index(drop=True)

df.loc[85:116] = df.loc[85:116].shift(1,axis=1)

In [464]:
mask = (df[4]!='')&(df[0]!='UNIDADES') & (df[1]!='OPÇÃO POR INGRESSO') & (df[2]!='PERÍODO')
df2 = df[mask]

In [465]:
df2[1] = np.where(df2[1]=='',df2[0],df2[1] )

df2 = df2[[1,2,3,4,5]]

In [466]:
df2 = df2.drop(index=[193,194,195])
df2[1] = df2[1].replace(df2[1].tolist()[-1], 'Física/Matemática/Mat. Aplicada e Computacional/Engenharia, Física/Física: Médica e Biomédica')

In [467]:
cols_rename = {
    1:'curso',
    2:'periodo',
    3:'ingressantes',
    4:'matriculados',
    5:'concluintes'
}
df2 = df2.rename(columns=cols_rename)

df2['ano_base'] = 2016

In [468]:
df2.to_excel('unicamp_alunos_2016_graduação.xlsx', index=False)

### Merge Data

In [725]:
df_2016 = pd.read_excel('dados/unicamp_alunos_2016_graduação.xlsx', dtype={'matriculados':int})
df_2017 = pd.read_excel('dados/unicamp_alunos_2017_graduação.xlsx', dtype={'matriculados':int})
df_2018 = pd.read_excel('dados/unicamp_alunos_2018_graduação.xlsx', dtype={'matriculados':int})



df_final = pd.concat([df_2016,df_2017], axis=0)
df_final = pd.concat([df_final,df_2018], axis=0)

df_final['curso'] = df_final['curso'].str.replace('\d','', regex=True).str.strip('()').str.strip().str.replace('  ',' ')
df_final['ingressantes'] = pd.to_numeric(df_final['ingressantes'],errors='coerce')
df_final['matriculados'] = pd.to_numeric(df_final['matriculados'],errors='coerce')
df_final['concluintes'] = pd.to_numeric(df_final['concluintes'],errors='coerce')




df_final['curso'] = df_final['curso'].str.replace('Letras – Licenciatura','Letras')

df_final = df_final.groupby(by=['curso','periodo','ano_base'], as_index=False).sum()

In [733]:
df_vagas=pd.read_excel('dados/unicamp_vestibular_2009_2018.xlsx')
df_vagas['curso'] = df_vagas['curso'].str.replace('\d','', regex=True).str.strip('()').str.strip().str.replace('  ',' ').str.replace('(','', regex=True).str.replace('CI','', regex=True).str.strip()

df_vagas['ano_base'] = df_vagas['ano_base'].astype(int)
df_vagas['curso'] = np.where(df_vagas['curso'].str.contains('Música'), 'Música', df_vagas['curso'])
df_vagas['vagas'] = pd.to_numeric(df_vagas['vagas'],errors='coerce')

mask = df_vagas['ano_base']>=2016
df_vagas = df_vagas[mask]

df_vagas['curso'] = df_vagas['curso'].str.replace('Física – Licenciatura','Licenciatura em Física').str.replace('Tec. em Análise e Desenvolvimento de Sistemas','Tec. em Análise e Desenvolvim. de Sistemas')
df_vagas['curso'] = df_vagas['curso'].str.replace('Tec. em Análise e Desenvolvimento de Sistemas','Tec. em Análise e Desenvolvim. de Sistemas')
df_vagas['curso'] = df_vagas['curso'].str.replace('Letras – Licenciatura','Letras')

df_vagas = df_vagas.groupby(by=['curso','periodo','ano_base'], as_index=False).sum()

In [734]:
df_merge = pd.merge(df_final,df_vagas,how='outer', on=['curso','periodo','ano_base'])

In [739]:
mask = df_merge['vagas'].notnull()

df_merge = df_merge[mask]

In [740]:
df_merge.to_excel('dados/unicam.xlsx', index=False)

,curso,periodo,ano_base,ingressantes,matriculados,concluintes,vagas
0,Arquitetura e Urbanismo,(N),2016,33.0,209.0,25.0,30.0
1,Arquitetura e Urbanismo,(N),2017,35.0,213.0,17.0,30.0
2,Arquitetura e Urbanismo,(N),2018,36.0,223.0,34.0,30.0
3,Artes Cênicas,(D),2016,26.0,112.0,25.0,25.0
4,Artes Cênicas,(D),2017,25.0,110.0,12.0,25.0
5,Artes Cênicas,(D),2018,27.0,123.0,25.0,25.0
6,Artes Visuais,(D),2016,42.0,179.0,37.0,30.0
7,Artes Visuais,(D),2017,40.0,169.0,29.0,30.0
8,Artes Visuais,(D),2018,40.0,172.0,39.0,30.0
9,Bacharelado em Administração,(N),2016,277.0,654.0,90.0,180.0
